In [7]:
import pickle

# load the complete association matrix of query-page (pre-constructed)
association_mat = pickle.load(open('data/all_question_article_ratings.pkl','rb'))

# load the complete text-id dictionary of queries (pre-constructed)
question_id = pickle.load(open('data/all_questions_dict.pkl','rb'))

# load the complete text-features dictionary of queries (pre-constructed)
question_features = pickle.load(open('data/all_questions_title.pkl','rb'))

# load the complete url-id dictionary of pages (pre-constructed)
page_id = pickle.load(open('data/infowave_allintents_dict.pkl','rb'))

# load the complete url-features dictionary of pages (pre-constructed)
page_features = pickle.load(open('data/infowave_allintents_title.pkl','rb'))


def sample_by_query_id():
    # sample the data by query-id, 
    sample_threhold = 30

    ratings = []
    dict_i_mapping={}

    # construct the dictionaries 
    dict_url={}
    dict_title_embed={}    
    
    #filter out query-id and page-id by association matrix
    for x,y in association_mat:
        if (x<sample_threhold): # filter by query id
            ratings.append((x,y))
            
    all_q_ids = [x for (x,y) in ratings]
    all_i_ids = [y for (x,y) in ratings]            
    
    #print(all_i_ids)
    
    icount = 0

    for k,v in page_id.items():
        #print(k,v)
        if ((v[0] in all_i_ids) and (k in page_features)):
            print('appending page id to dictionary')
            dict_url[k]=v[0]
            dict_title_embed[k]=page_features[k]
            
            if v[0] not in dict_i_mapping:
                dict_i_mapping[v[0]]=icount
                icount+=1
                
    #print(dict_i_mapping)
    
    new_ratings=[]

    # remap the page id to the sampled new ids 
    for x,y in ratings:
        new_ratings.append((x,dict_i_mapping[y]))
    
    
    # replace the rating association matrix with the one with new ID-mappings
    ratings = new_ratings

    # construct the dictionaries 
    dict_url={}
    dict_url_embed={}    

#     dict_question={}
#     dict_question_embed={}

    for k,v in page_id.items():
        if (v[0] in all_i_ids) and (k in page_features):

            dict_url[k]=dict_i_mapping[v[0]]
            dict_url_embed[k]=page_features[k]    
    
    pickle.dump(ratings,open('data/sample_by_page_questions_article_ratings.pkl','wb'))
    pickle.dump(dict_url,open('data/sample_by_page_page_dict.pkl','wb'))
    pickle.dump(dict_title_embed,open('data/sample_by_page_page_features.pkl','wb'))


def sample_by_page_id():
    # sample the data by page-id, here we limit a number of page ids,  only keep the page < 10
    sample_threhold = 10

    ratings=[]
    dict_question={}
    dict_question_embed={}

    dict_q_mapping={}
    dict_url={}
    dict_title={}

    for x,y in association_mat:
        if (y<sample_threhold): # filter by page id
            ratings.append((x,y))

            #print(x,y)

    all_q_ids = [x for (x,y) in ratings]
    all_i_ids = [y for (x,y) in ratings]

    icount = 0

    for k,v in question_id.items():
        if (v in all_q_ids) and (k in question_features):

            dict_question[k]=v
            dict_question_embed[k]=question_features[k]
            if v not in dict_q_mapping:
                dict_q_mapping[v]=icount
                icount+=1

    
    new_ratings=[]

    for x,y in ratings:
        new_ratings.append((dict_q_mapping[x],y))

    ratings = new_ratings

    dict_question={}
    dict_question_embed={}

    for k,v in question_id.items():
        if (v in all_q_ids) and (k in question_features):

            dict_question[k]=dict_q_mapping[v]
            dict_question_embed[k]=question_features[k]

    pickle.dump(ratings,open('data/sample_by_question_questions_article_ratings.pkl','wb'))
    pickle.dump(dict_question,open('data/sample_by_question_questions_dict.pkl','wb'))
    pickle.dump(dict_question_embed,open('data/sample_by_question_questions_features.pkl','wb'))

    
sample_by_page_id()

In [2]:
page_id = pickle.load(open('data/sample_by_page_page_dict.pkl','rb'))
page_features = pickle.load(open('data/sample_by_page_page_features.pkl','rb'))       

In [3]:
page_id

{'3 Rollovers, 1 was Roth but went into a pretax account. How to fix?': 0,
 'can client fax/msg rollover certificaiton letter?': 1,
 'Can client have a rollover wired into Vanguard?': 2,
 'Can client move money from ESA to another type of account?': 3,
 'Can client return Indirect rollover in increments?': 4,
 'can rollover go into any acct?': 5,
 'can send OATS with the rollover check?': 6,
 'Can we accept a direct rollover check without an account number on it?': 7,
 'can we fax in rollover cert letter': 8,
 'can we fill out the tsp 60 form?': 9,
 'can we rollover a 401k in kind': 10,
 'client did an outgoing rollover and said there was a from we were supposed to sign and we didnt do it': 11,
 'Do we need to get the name change done first?': 12,
 'How can we accept a name variation on a rollover check?': 13,
 'We sent the D/r check to the TSP but did not complete the attached TSP form 60 according to what they received which is preventing them from accepting the check, how to fix the

In [23]:
a='http://oedesktop.vanguard.com/us/content/InfoWave/AssocHbk/ProbRes/ProblemResolutionMatrices.jsp'
a in page_features

True